In [81]:
import pandas as pd
import os

In [82]:
df = pd.concat([pd.read_csv("data/firearm-charges-2018.csv"),
                pd.read_csv("data/firearm-charges-2013.csv"),
                pd.read_csv("data/firearm-charges-2007.csv").dropna(subset=["Year"])])

df = df[["Year","Charges","Offense","Table"]]
df.head()

,Year,Charges,Offense,Table
0,2013,4,VIOLATING PISTOL PERMITTING REQUIREMENTS (CGS ...,1.0
1,2014,3,VIOLATING PISTOL PERMITTING REQUIREMENTS (CGS ...,1.0
2,2015,5,VIOLATING PISTOL PERMITTING REQUIREMENTS (CGS ...,1.0
3,2016,7,VIOLATING PISTOL PERMITTING REQUIREMENTS (CGS ...,1.0
4,2017,10,VIOLATING PISTOL PERMITTING REQUIREMENTS (CGS ...,1.0


In [83]:
df[(df["Year"] < 2008)&(df["Table"] == 2)]["Offense"].unique()

array([], dtype=object)

In [84]:
def asciistr(badstr):
    if pd.isnull(badstr): return badstr
    return ''.join([i if ord(i) < 128 else '' for i in badstr]).strip().replace("/"," ")
df["ascii_offense"] = df["Offense"].apply(asciistr)

In [85]:
import re
def statute_code(offense_str):
    if pd.isnull(offense_str): return ""
    return re.search(".* \((.*)\)",asciistr(offense_str)).group(1)

statute_code(df.iloc[20]["Offense"])


'CGS  29-35(a)'

In [86]:
df["Statute"] = df["Offense"].apply(statute_code)
df.head()

,Year,Charges,Offense,Table,ascii_offense,Statute
0,2013,4,VIOLATING PISTOL PERMITTING REQUIREMENTS (CGS ...,1.0,VIOLATING PISTOL PERMITTING REQUIREMENTS (CGS ...,CGS 29-28
1,2014,3,VIOLATING PISTOL PERMITTING REQUIREMENTS (CGS ...,1.0,VIOLATING PISTOL PERMITTING REQUIREMENTS (CGS ...,CGS 29-28
2,2015,5,VIOLATING PISTOL PERMITTING REQUIREMENTS (CGS ...,1.0,VIOLATING PISTOL PERMITTING REQUIREMENTS (CGS ...,CGS 29-28
3,2016,7,VIOLATING PISTOL PERMITTING REQUIREMENTS (CGS ...,1.0,VIOLATING PISTOL PERMITTING REQUIREMENTS (CGS ...,CGS 29-28
4,2017,10,VIOLATING PISTOL PERMITTING REQUIREMENTS (CGS ...,1.0,VIOLATING PISTOL PERMITTING REQUIREMENTS (CGS ...,CGS 29-28


In [87]:
statutes = df["Statute"].unique()


In [88]:
def plot_title(ascii_str):
    if pd.isnull(ascii_str): return ascii_str
    return ascii_str[:ascii_str.index("(")].lower().capitalize().strip()
df["Title"] = df["ascii_offense"].apply(plot_title)

In [89]:
def fix(val):
    return str(int(str(val).strip("*").strip().replace(",","")))


print fix("199*")
print fix("*2100")
print fix("*2100*")

199
2100
2100


In [90]:
def fix_col(col_name):
    df[col_name] = df[col_name].apply(fix).astype(int)
fix_col("Year")
fix_col("Charges")
#fix_col("Convictions")
#fix_col("Nolle")

In [91]:
def exclude_01(tmpdf):
    tmpdf = tmpdf.copy().reset_index()
    # Not sure why 2001 numbers are so low. Excluding for now.
    return tmpdf[tmpdf["Year"] > 2001]


In [92]:
df.dtypes

Year               int64
Charges            int64
Offense           object
Table            float64
ascii_offense     object
Statute           object
Title             object
dtype: object

In [93]:
df = df[["Year","Charges","Table","Title","Statute"]]

In [94]:
df.head()

,Year,Charges,Table,Title,Statute
0,2013,4,1.0,Violating pistol permitting requirements,CGS 29-28
1,2014,3,1.0,Violating pistol permitting requirements,CGS 29-28
2,2015,5,1.0,Violating pistol permitting requirements,CGS 29-28
3,2016,7,1.0,Violating pistol permitting requirements,CGS 29-28
4,2017,10,1.0,Violating pistol permitting requirements,CGS 29-28


In [95]:
offenses = df["Title"].unique()

In [96]:
# def from_peak(df):
#     ret = df.copy()
    
#     peak = float(ret.loc[ret.index[0]]["Charges"])
#     latest = float(ret.loc[ret.index[-1]]["Charges"])
    
#     return (peak - latest) * 100 / peak
    

In [97]:
def statute_df(statute):
    return df[df["Statute"] == statute].set_index("Year").sort_index()

In [98]:
def off_df(offense):
    ret = df[df["Title"] == offense].set_index("Year").sort_index()
    
#     ret["from_peak"] = from_peak(ret)
    
    return ret

off_df(offenses[10])

,Charges,Table,Title,Statute
Year,,,,
2008,17,1.0,Violating loaded firearm safe storage requirem...,CGS 29-37i
2009,7,1.0,Violating loaded firearm safe storage requirem...,CGS 29-37i
2010,26,1.0,Violating loaded firearm safe storage requirem...,CGS 29-37i
2011,20,1.0,Violating loaded firearm safe storage requirem...,CGS 29-37i
2012,12,1.0,Violating loaded firearm safe storage requirem...,CGS 29-37i
2013,10,1.0,Violating loaded firearm safe storage requirem...,CGS 29-37i
2014,12,1.0,Violating loaded firearm safe storage requirem...,CGS 29-37i
2015,12,1.0,Violating loaded firearm safe storage requirem...,CGS 29-37i
2016,7,1.0,Violating loaded firearm safe storage requirem...,CGS 29-37i


In [99]:
import matplotlib
#matplotlib.use( "agg" )
# matplotlib.use( "macosx" )

%matplotlib inline
# matplotlib.style.use('ggplot')
matplotlib.style.use('mirror00')
import matplotlib.pyplot as plt

In [60]:
def plot_offense(offdf,title="",subfolder="",figsize=(6,4)):
    
    #offdf.set_index("Year")
    
    print offdf.index
    
    tmpdf = exclude_01(offdf.copy())
    fname = tmpdf["Title"][tmpdf.index[0]]
    ax = tmpdf["Charges"].plot.bar(figsize=figsize)
    ax.set_xlabel("")
    #ax.set_title(off_df["title"].iloc[0])
    ax.set_title(title)
    
    ax.set_xticklabels(tmpdf["Year"])
        
    plt.tight_layout()
    
    plt.savefig(os.path.join("img", subfolder, fname + ".png"),
                bbox_inches="tight")
    
    plt.close()


plot_offense(off_df(offenses[10]))

Int64Index([2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017], dtype='int64', name=u'Year')


In [61]:
for statute in statutes:
    tmpdf = statute_df(statute)
    if tmpdf["Charges"].max() > 100:
#         print off
        plot_offense(tmpdf, subfolder="over-50",figsize=(6,4))
    try:
        plot_offense(tmpdf)
    except:
        print tmpdf

Int64Index([2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012,
            2013, 2014, 2015, 2016, 2017],
           dtype='int64', name=u'Year')
Int64Index([2005, 2006, 2008, 2009, 2010, 2013], dtype='int64', name=u'Year')
Int64Index([2001, 2002, 2002, 2003, 2003, 2004, 2004, 2005, 2005, 2006, 2007,
            2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017],
           dtype='int64', name=u'Year')
Int64Index([2001, 2002, 2002, 2003, 2003, 2004, 2004, 2005, 2005, 2006, 2007,
            2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017],
           dtype='int64', name=u'Year')
Int64Index([2003, 2008, 2009, 2012, 2015, 2017], dtype='int64', name=u'Year')
Int64Index([2002, 2003, 2004, 2005, 2006, 2007, 2008, 2011, 2012, 2014, 2015,
            2016],
           dtype='int64', name=u'Year')
Int64Index([2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011,
            2012, 2013, 2014, 2015, 2016, 2017],
           dtype='int64',

In [62]:
# for off in offenses:
#     tmpdf = off_df(off)
#     if tmpdf["Charges"].max() > 100:
#         print off
#         plot_offense(tmpdf, subfolder="over-50",figsize=(6,4))
#     try:
#         plot_offense(tmpdf)
#     except:
#         print tmpdf

In [100]:
def prev_avg(tmpdf):
    tmpdf = tmpdf.copy().reset_index()
    years = tmpdf["Year"]
    years_to_average =  filter(lambda x: x != 2017, years)
    good_year = lambda x: x != 2017
    return tmpdf[tmpdf["Year"].apply(good_year)]["Charges"].mean()
    
    
prev_avg(off_df(offenses[10]))

13.666666666666666

In [107]:
rows = []

for statute in statutes:
    tmpdf = statute_df(statute)
    if tmpdf["Charges"].max() > 0 and 2017 in tmpdf.index:

    #if tmpdf["Charges"].max() > 300 and 2017 in tmpdf.index:
        tmpdf = tmpdf.sort_values(by="Charges",ascending=False)
        peak_row = tmpdf.reset_index().iloc[0]

        if 2017 in tmpdf.index:
            count2017 = tmpdf.loc[2017]["Charges"]
        else:
            count2017 = 0
            
        previous_average = prev_avg(tmpdf)
        diff_avg = previous_average - count2017
        #diff_avg_pct = diff_avg / previous_average
        obj = {
                "statute":statute,
                "offense":peak_row["Title"],
                "peak_year":peak_row["Year"],
                "peak":peak_row["Charges"],
                "2017":count2017,
                "diff": peak_row["Charges"] - count2017,
                "prev_avg": int(previous_average.round()),
                "diff_avg": diff_avg,
                "table":tmpdf.iloc[0]["Table"]
        }
        
        print obj 
        
        rows.append(obj)
        
top_df = pd.DataFrame(rows).set_index("statute")
top_df["diff_pct"] = top_df["diff"] * 100 / top_df["peak"]
top_df["diff_avg_pct"] = top_df["diff_avg"] * 100 / top_df["prev_avg"]

#top_df["diff_pct"] = top_df["diff_pct"].apply(lambda x: round(x,1))
#top_df["diff_avg_pct"] = top_df["diff_avg_pct"].apply(lambda x: round(x,1))

top_df["Sentence"] = top_df.index + " charges dropped " + top_df["diff_pct"].astype(str) + " percent "\
+ " from a " + top_df["peak_year"].astype(str) + " peak of " + top_df["peak"].astype(str)\
+ " to " + top_df["2017"].astype(str) + " in 2017."

top_df["Sentence2"] = top_df.index + " charges dropped " + top_df["diff_avg_pct"].astype(str) + " percent "\
+ " from a 15-year average of " + top_df["prev_avg"].astype(str)\
+ " to " + top_df["2017"].astype(str) + " in 2017."

top_df["Sentence3"] = "<strong>" + top_df.index + " charges</strong> dropped " + top_df["diff_avg_pct"].astype(str) + " percent "\
+ " from a 15-year average of " + top_df["prev_avg"].astype(str)\
+ " to " + top_df["2017"].astype(str) + " in 2017."\
+ " Charges for this offense peaked at " + top_df["peak"].astype(str) + " in " + top_df["peak_year"].astype(str) + "."


top_df

        

{'statute': 'CGS  29-28', 'peak': 10, 'prev_avg': 5, 'diff': 0, 'diff_avg': -4.9333333333333336, 'offense': 'Violating pistol permitting requirements', '2017': 10, 'table': 1.0, 'peak_year': 2010}
{'statute': 'CGS  29-33', 'peak': 206, 'prev_avg': 64, 'diff': 127, 'diff_avg': -15.047619047619051, 'offense': 'Illegally transferring a pistol or revolver', '2017': 79, 'table': 1.0, 'peak_year': 2011}
{'statute': 'CGS  29-34(a)', 'peak': 2, 'prev_avg': 1, 'diff': 0, 'diff_avg': -0.80000000000000004, 'offense': 'Making false statements related to a pistol or revolver transfer', '2017': 2, 'table': 1.0, 'peak_year': 2012}
{'statute': 'CGS  29-35(a)', 'peak': 681, 'prev_avg': 581, 'diff': 32, 'diff_avg': -68.25, 'offense': 'Carrying a pistol without a permit', '2017': 649, 'table': 1.0, 'peak_year': 2008}
{'statute': 'CGS  29-35(b)', 'peak': 112, 'prev_avg': 8, 'diff': 0, 'diff_avg': -103.86666666666667, 'offense': 'Failing to carry a pistol permit', '2017': 112, 'table': 1.0, 'peak_year': 20

,2017,diff,diff_avg,offense,peak,peak_year,prev_avg,table,diff_pct,diff_avg_pct,Sentence,Sentence2,Sentence3
statute,,,,,,,,,,,,,
CGS 29-28,10,0,-4.93333,Violating pistol permitting requirements,10,2010,5,1.0,0,-98.6667,CGS 29-28 charges dropped 0.0 percent from a...,CGS 29-28 charges dropped -98.6666666667 perc...,<strong>CGS 29-28 charges</strong> dropped -9...
CGS 29-33,79,127,-15.0476,Illegally transferring a pistol or revolver,206,2011,64,1.0,61.6505,-23.5119,CGS 29-33 charges dropped 61.6504854369 perce...,CGS 29-33 charges dropped -23.5119047619 perc...,<strong>CGS 29-33 charges</strong> dropped -2...
CGS 29-34(a),2,0,-0.8,Making false statements related to a pistol or...,2,2012,1,1.0,0,-80,CGS 29-34(a) charges dropped 0.0 percent fro...,CGS 29-34(a) charges dropped -80.0 percent f...,<strong>CGS 29-34(a) charges</strong> dropped...
CGS 29-35(a),649,32,-68.25,Carrying a pistol without a permit,681,2008,581,1.0,4.69897,-11.747,CGS 29-35(a) charges dropped 4.69897209985 pe...,CGS 29-35(a) charges dropped -11.7469879518 p...,<strong>CGS 29-35(a) charges</strong> dropped...
CGS 29-35(b),112,0,-103.867,Failing to carry a pistol permit,112,2017,8,1.0,0,-1298.33,CGS 29-35(b) charges dropped 0.0 percent fro...,CGS 29-35(b) charges dropped -1298.33333333 p...,<strong>CGS 29-35(b) charges</strong> dropped...
CGS 29-36,115,0,-27.1875,"Altering firearm identification mark, number, ...",115,2017,88,1.0,0,-30.8949,CGS 29-36 charges dropped 0.0 percent from a...,CGS 29-36 charges dropped -30.8948863636 perc...,<strong>CGS 29-36 charges</strong> dropped -3...
CGS 29-37e,6,16,0.727273,Making a false statement related to transferri...,22,2009,7,1.0,72.7273,10.3896,CGS 29-37e charges dropped 72.7272727273 perc...,CGS 29-37e charges dropped 10.3896103896 perc...,<strong>CGS 29-37e charges</strong> dropped 1...
CGS 29-37i,13,13,0.666667,Violating loaded firearm safe storage requirem...,26,2010,14,1.0,50,4.7619,CGS 29-37i charges dropped 50.0 percent from...,CGS 29-37i charges dropped 4.7619047619 perce...,<strong>CGS 29-37i charges</strong> dropped 4...
CGS 29-37j(c),1,1,1,Purchase or soliciting purchase of a firearm f...,2,2005,2,NaN,50,50,CGS 29-37j(c) charges dropped 50.0 percent f...,CGS 29-37j(c) charges dropped 50.0 percent f...,<strong>CGS 29-37j(c) charges</strong> droppe...


In [108]:
for s in top_df["Sentence"]: print s + "\n"

CGS  29-28 charges dropped 0.0 percent  from a 2010 peak of 10 to 10 in 2017.

CGS  29-33 charges dropped 61.6504854369 percent  from a 2011 peak of 206 to 79 in 2017.

CGS  29-34(a) charges dropped 0.0 percent  from a 2012 peak of 2 to 2 in 2017.

CGS  29-35(a) charges dropped 4.69897209985 percent  from a 2008 peak of 681 to 649 in 2017.

CGS  29-35(b) charges dropped 0.0 percent  from a 2017 peak of 112 to 112 in 2017.

CGS  29-36 charges dropped 0.0 percent  from a 2017 peak of 115 to 115 in 2017.

CGS  29-37e charges dropped 72.7272727273 percent  from a 2009 peak of 22 to 6 in 2017.

CGS  29-37i charges dropped 50.0 percent  from a 2010 peak of 26 to 13 in 2017.

CGS  29-37j(c) charges dropped 50.0 percent  from a 2005 peak of 2 to 1 in 2017.

CGS  53-202(b) charges dropped 77.7777777778 percent  from a 2002 peak of 9 to 2 in 2017.

CGS  53-202(c) charges dropped 66.6666666667 percent  from a 2004 peak of 9 to 3 in 2017.

CGS  53-202b(a)(1) charges dropped 81.8181818182 percent  

In [109]:
for s in top_df["Sentence2"]: print s+ "\n"

CGS  29-28 charges dropped -98.6666666667 percent  from a 15-year average of 5 to 10 in 2017.

CGS  29-33 charges dropped -23.5119047619 percent  from a 15-year average of 64 to 79 in 2017.

CGS  29-34(a) charges dropped -80.0 percent  from a 15-year average of 1 to 2 in 2017.

CGS  29-35(a) charges dropped -11.7469879518 percent  from a 15-year average of 581 to 649 in 2017.

CGS  29-35(b) charges dropped -1298.33333333 percent  from a 15-year average of 8 to 112 in 2017.

CGS  29-36 charges dropped -30.8948863636 percent  from a 15-year average of 88 to 115 in 2017.

CGS  29-37e charges dropped 10.3896103896 percent  from a 15-year average of 7 to 6 in 2017.

CGS  29-37i charges dropped 4.7619047619 percent  from a 15-year average of 14 to 13 in 2017.

CGS  29-37j(c) charges dropped 50.0 percent  from a 15-year average of 2 to 1 in 2017.

CGS  53-202(b) charges dropped 4.16666666667 percent  from a 15-year average of 2 to 2 in 2017.

CGS  53-202(c) charges dropped 37.5 percent  from 

In [ ]:
for s in top_df["Sentence3"]: print s + "\n"

In [105]:
# yearly totals

fonly_df = df[df["Table"]!=2].groupby("Year").agg({"Charges":sum})
fonly_df["Title"] = "All firearm-only"
fonly_df.to_csv("data/firearm-only.csv")
fonly_df

,Charges,Title
Year,,
2001,441,All firearm-only
2002,2069,All firearm-only
2003,2293,All firearm-only
2004,2109,All firearm-only
2005,2515,All firearm-only
2006,2488,All firearm-only
2007,1968,All firearm-only
2008,2821,All firearm-only
2009,2604,All firearm-only


In [110]:
prev_avg(fonly_df)

2392.75

In [111]:
plot_offense(fonly_df,title="Firearm-only charges",figsize=(6,4))

Int64Index([2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011,
            2012, 2013, 2014, 2015, 2016, 2017],
           dtype='int64', name=u'Year')


In [112]:
maybe_df = df[df["Table"]==2].groupby("Year").agg({"Charges":sum})
maybe_df["Title"] = "All potential firearm-involved"
plot_offense(maybe_df,title="Potentially firearm",figsize=(6,4))

Int64Index([2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011,
            2012, 2013, 2014, 2015, 2016, 2017],
           dtype='int64', name=u'Year')


In [113]:
all_df = df[df["Table"]==2].groupby("Year").agg({"Charges":sum})
all_df["Title"] = "All offenses"
plot_offense(all_df,figsize=(6,4))

Int64Index([2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011,
            2012, 2013, 2014, 2015, 2016, 2017],
           dtype='int64', name=u'Year')


In [114]:
df.groupby("Year").agg({"Charges":sum}).to_csv("data/potential-firearm-charges.csv")
df.groupby("Year").agg({"Charges":sum})

,Charges
Year,
2001,877
2002,4104
2003,4472
2004,4246
2005,4637
2006,4650
2007,3739
2008,5119
2009,4650


In [115]:
df[df["Table"]==2].groupby("Year").agg({"Charges":sum})

,Charges
Year,
2001,436
2002,2035
2003,2179
2004,2137
2005,2122
2006,2162
2007,1771
2008,2298
2009,2046


In [116]:
df.groupby("Year").agg({"Charges":sum})

,Charges
Year,
2001,877
2002,4104
2003,4472
2004,4246
2005,4637
2006,4650
2007,3739
2008,5119
2009,4650


In [117]:
df.to_csv("data/combined-firearm-charges.csv",index=False)

In [118]:
df[df["Table"] == 2]["Title"].unique()

array(['Possessing a weapon in a vehicle', 'Carrying a dangerous weapon',
       'Possessing a firearm or deadly weapon on school grounds',
       'Possess firearm or deadly weapon on school ground'], dtype=object)

In [119]:
df.head()


,Year,Charges,Table,Title,Statute
0,2013,4,1.0,Violating pistol permitting requirements,CGS 29-28
1,2014,3,1.0,Violating pistol permitting requirements,CGS 29-28
2,2015,5,1.0,Violating pistol permitting requirements,CGS 29-28
3,2016,7,1.0,Violating pistol permitting requirements,CGS 29-28
4,2017,10,1.0,Violating pistol permitting requirements,CGS 29-28
